In [1]:
# Packages for data analysis
import pandas as pd
import numpy as np

In [2]:
pip install tweet-preprocessor

Note: you may need to restart the kernel to use updated packages.


In [3]:
df=pd.read_csv('data/100tweets_no_other.csv', sep=';')
print(df.shape)
df.head()

(84, 18)


,Column1,...1,Row.names,rowid,lat2,lon2,user_id,Column2,text2,retweeted,lang,id,created_at,noChar,iso_lang_1,prob_1,text1,classname
0,4,53090,93493,319774,49.302954,7.466072,434326976,NaN,"Not good, not bad...disappointing",False,en,"1,19E+18",29.10.2019 16:55,34,en,0.881157,"Not good, not bad...disappointing\r\n#MartinaP...",home
1,8,9104,145546,679738,49.611273,6.12770012,1298568337,NaN,"Monday, June , : Spaghetti Bolognese",False,it,"3,42E+17",03.06.2013 11:22,37,en,0.914869,"Monday, June 3, 2013: Spaghetti Bolognese http...",home
2,24,23743,25333,105500,49.658195,6.14052043,28345806,NaN,Where are the people who left on the boat?? Do...,False,en,"5,63E+17",05.02.2015 21:53,70,en,0.964202,Where are the people who left on the boat?? Do...,home
3,38,55020,97806,347149,63.833300,20.25,456045381,NaN,Christmas cosiness. Merry very. |,False,sv,"8,13E+17",24.12.2016 21:21,34,en,0.924872,Christmas cosiness. Merry very. | #christmasev...,home
4,83,49773,82682,277614,49.499172,6.326877099999999,339795921,NaN,Again... Who'll find my husband?,False,en,"8,05E+17",04.12.2016 14:08,33,en,0.968227,Again... Who'll find my husband? https://t.co/...,home


In [4]:
import preprocessor as prep

In [5]:
## Set options for the tweet-preprocessor
prep.set_options(prep.OPT.URL, prep.OPT.RESERVED, prep.OPT.EMOJI, prep.OPT.SMILEY)

## Clean text and remove #,@ symbols
def clean_tweet(text):
    text = prep.clean(text)
    table = str.maketrans('','','#@')
    return text.translate(table)

In [6]:
df['text_processed'] = df['text1'].apply(lambda x: clean_tweet(x))

In [7]:
print(df.shape)
df.head()

(84, 19)


,Column1,...1,Row.names,rowid,lat2,lon2,user_id,Column2,text2,retweeted,lang,id,created_at,noChar,iso_lang_1,prob_1,text1,classname,text_processed
0,4,53090,93493,319774,49.302954,7.466072,434326976,NaN,"Not good, not bad...disappointing",False,en,"1,19E+18",29.10.2019 16:55,34,en,0.881157,"Not good, not bad...disappointing\r\n#MartinaP...",home,"Not good, not bad...disappointing MartinaPadbe..."
1,8,9104,145546,679738,49.611273,6.12770012,1298568337,NaN,"Monday, June , : Spaghetti Bolognese",False,it,"3,42E+17",03.06.2013 11:22,37,en,0.914869,"Monday, June 3, 2013: Spaghetti Bolognese http...",home,"Monday, June 3, 2013: Spaghetti Bolognese"
2,24,23743,25333,105500,49.658195,6.14052043,28345806,NaN,Where are the people who left on the boat?? Do...,False,en,"5,63E+17",05.02.2015 21:53,70,en,0.964202,Where are the people who left on the boat?? Do...,home,Where are the people who left on the boat?? Do...
3,38,55020,97806,347149,63.833300,20.25,456045381,NaN,Christmas cosiness. Merry very. |,False,sv,"8,13E+17",24.12.2016 21:21,34,en,0.924872,Christmas cosiness. Merry very. | #christmasev...,home,Christmas cosiness. Merry very. | christmaseve...
4,83,49773,82682,277614,49.499172,6.326877099999999,339795921,NaN,Again... Who'll find my husband?,False,en,"8,05E+17",04.12.2016 14:08,33,en,0.968227,Again... Who'll find my husband? https://t.co/...,home,Again... Who'll find my husband? ?meowclank


In [8]:
# transforming column with tweets to the list
sentences = list(df['text_processed'].values) # text will be the input to the model

In [10]:
# Do not forget to use GPU in Colab
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [12]:
from transformers import pipeline

In [14]:
# Applying classifier to the list of tweets
activity_classifier = pipeline("zero-shot-classification", model = 'facebook/bart-large-mnli')
activity_result = activity_classifier(list(sentences), candidate_labels = [
'Sleep waking up', 'Baby infant family', 'Reading books', 'TV YouTube Netflix', 'Dish recipy kitchen',

'Campus school university education study','Work job','Training secondment traineeship',
'Office lunch colleagues department director','Business industry research conferences and forums', 

'Outdoor sports recreation','Nature landscape wildlife','Walk hiking','Weather season rain cold',

'Delays passenger arrival departure','Cycling bikes scooters','Airport waiting lounge','Car petrol traffic',

'Entertainment','Religion','Groceries and shopping','Healthcare and well-being',
'Fitness club', 'Services brand retail','Café restaurants','City sightseeing','Art shows performances'],)
activity_result[10]

{'sequence': 'One more sunny day in beautiful VilledeNice France',
 'labels': ['City sightseeing',
  'Entertainment',
  'Nature landscape wildlife',
  'Healthcare and well-being',
  'Café restaurants',
  'Outdoor sports recreation',
  'Dish recipy kitchen',
  'Walk hiking',
  'Office lunch colleagues department director',
  'Car petrol traffic',
  'Services brand retail',
  'Fitness club',
  'Baby infant family',
  'Delays passenger arrival departure',
  'Cycling bikes scooters',
  'Campus school university education study',
  'Sleep waking up',
  'Art shows performances',
  'TV YouTube Netflix',
  'Training secondment traineeship',
  'Groceries and shopping',
  'Airport waiting lounge',
  'Business industry research conferences and forums',
  'Weather season rain cold',
  'Work job',
  'Reading books',
  'Religion'],
 'scores': [0.14973075687885284,
  0.11582930386066437,
  0.10915405303239822,
  0.07279437780380249,
  0.05855735018849373,
  0.05447577312588692,
  0.049717407673597336

In [15]:
np.argmax(activity_result[0]['scores']) 

0

In [16]:
# Leaving only first prediction for validation
y_activity_pred = [result['labels'][np.argmax(result['scores'])] for result in activity_result]
y_activity_pred[10]

'City sightseeing'

In [17]:
# Replacing predictions with general categories for validation
for i in range(len(y_activity_pred)):
  
    if y_activity_pred[i] == 'Sleep waking up':
        y_activity_pred[i] = 'home'    
    if y_activity_pred[i] == 'Baby infant family':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'Reading books':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'TV YouTube Netflix':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'Dish recipy kitchen':
        y_activity_pred[i] = 'home' 

    if y_activity_pred[i] == 'Campus school university education study':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Work job':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Training secondment traineeship':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Office lunch colleagues department director':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Business industry research conferences and forums':
        y_activity_pred[i] = 'work'

    if y_activity_pred[i] == 'Outdoor sports recreation':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Nature landscape wildlife':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Weather season rain cold':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Walk hiking':
        y_activity_pred[i] = 'outdoors'

    if y_activity_pred[i] == 'Delays passenger arrival departure':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Cycling bikes scooters':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Airport waiting lounge':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Car petrol traffic':
        y_activity_pred[i] = 'transport'
        
    if y_activity_pred[i] == 'Entertainment':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Religion':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Groceries and shopping':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Healthcare and well-being':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Fitness club':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Services brand retail':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Café restaurants':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'City sightseeing':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Theatre shows performances':
        y_activity_pred[i] = 'services'
  
# print list
print(y_activity_pred)

['home', 'home', 'transport', 'services', 'services', 'home', 'outdoors', 'services', 'services', 'services', 'services', 'services', 'outdoors', 'outdoors', 'outdoors', 'services', 'services', 'services', 'services', 'services', 'services', 'home', 'services', 'services', 'transport', 'services', 'services', 'services', 'services', 'work', 'work', 'services', 'services', 'services', 'services', 'home', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'work', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'services', 'outdoors', 'services', 'transport', 'outdoors', 'services', 'transport', 'services', 'transport', 'transport', 'outdoors', 'transport', 'transport', 'transport', 'transport', 'work', 'services', 'services', 'work', 'work', 'work', 'services', 'services', 'services', 'work', 'outdoors', 'services', 'work', 'home', 'work', 'outdoors']


In [18]:
y_activity_true = list(df['classname'].values) # true labels (annotated by human)

In [19]:
from sklearn.metrics import classification_report, confusion_matrix

In [20]:
# Classification report for the general categories (1 row left unannotated, will be fixed)
print(classification_report(y_activity_true, y_activity_pred))

              precision    recall  f1-score   support

        home       0.50      0.50      0.50         6
    outdoors       0.44      0.40      0.42        10
    services       0.69      0.81      0.75        42
   transport       0.80      0.67      0.73        12
        work       0.70      0.50      0.58        14

    accuracy                           0.67        84
   macro avg       0.63      0.58      0.60        84
weighted avg       0.67      0.67      0.66        84



In [31]:
# Transforming list of predictions back to dataframe
df_result = pd.DataFrame(activity_result)

In [32]:
# Checking predictions
df_result.head()

,sequence,labels,scores
0,(Senior) Digital Product Designer in #Leipzig ...,"[Work job training, Discussion, Gym, Culture a...","[0.1974041610956192, 0.10330083966255188, 0.08..."
1,Jugging down my 2nd Liter of coffee for today....,"[Café restaurants drinks, Discussion, Culture ...","[0.11716744303703308, 0.09599005430936813, 0.0..."
2,Happy Weekend ?? #Metz #Lorraine https://t.co/...,"[City sightseeing, Culture and entertainment, ...","[0.18392527103424072, 0.1045927107334137, 0.06..."
3,"Good morning, Vancouver! :) #Sunrise 07:13, no...","[Discussion, Departures delays arrivals, City ...","[0.08371323347091675, 0.07710594683885574, 0.0..."
4,"Not good, not bad...disappointing\r\n#MartinaP...","[Discussion, Reading and listening, Departures...","[0.11798880994319916, 0.08938255906105042, 0.0..."


In [33]:
# Saving prediction results to csv
df_result.to_csv('result.csv')